In [13]:
import pandas as pd

# Revise the movie dataset to add extended data

In [14]:
ratings = pd.read_csv('finalDB/ratings.csv', header = 0) 
stats = ratings.groupby(['movieID'])['rating'].agg(['count','sum']).to_dict()    #create a dictionary for count and sum

Rewrite the movies data set with the genres, description, sum, count of voters

In [9]:
movies = pd.read_csv('finalDB/movies.csv', header = 0)         #read file 
extended_movies = pd.read_csv('finalDB/movies_preliminary.csv', header = 0)         #read file 
movies['genre'] = movies.apply(lambda _: '', axis=1)          #newcolumn for genre
movies['description'] = movies.apply(lambda _: '', axis=1)          #newcolumn for description
movies['total_ratings'] = movies.apply(lambda _: '', axis=1)            #newcolumn for sum of all votes
movies['rating_count'] = movies.apply(lambda _: '', axis=1)          #newcolumn for number of voters

extended = pd.Series(extended_movies.index,index = extended_movies.imdb_id).to_dict() #dictionary of imdb_id: row in extended dataset 

for i in range(len(movies)):
    imdbid = movies.iloc[i,2]
    row = extended[imdbid]
    movies.iloc[i,5] = extended_movies.iloc[row,3]                    #retrieve the genre for that movie
    movies.iloc[i,6] = extended_movies.iloc[row,8]                    #retrieve the description for that movie
    if imdbid in stats['count']:                                                 #check if the movieid had any reviews added for it
        movies.iloc[i,7] = stats['sum'][imdbid]                        #retrieving the number of voters in the ratings dataset for that movie
        movies.iloc[i,8] = stats['count'][imdbid]                          #retrieving the sum of votes in the ratings dataset for that movie
movies

,movie_id,title,imdb_id,imdb_title_id,year,genre,description,total_ratings,rating_count
0,1,Toy Story,114709,tt114709,1995,"Animation, Adventure, Comedy",A cowboy doll is profoundly threatened and jea...,7753.5,2000
1,2,Jumanji,113497,tt113497,1995,"Adventure, Comedy, Family",When two kids find and play a magical board ga...,5445.5,1734
2,3,Grumpier Old Men,113228,tt113228,1995,"Comedy, Romance",John and Max resolve to save their beloved bai...,1836,609
3,4,Waiting to Exhale,114885,tt114885,1995,"Comedy, Drama, Romance","Based on Terry McMillan's novel, this film fol...",490,174
4,5,Father of the Bride Part II,113041,tt113041,1995,"Comedy, Family, Romance",George Banks must deal not only with the pregn...,1820.5,614
...,...,...,...,...,...,...,...,...,...
9412,203519,Fast & Furious Presents: Hobbs & Shaw,6806448,tt6806448,2019,"Action, Adventure, Thriller","Lawman Luke Hobbs (Dwayne ""The Rock"" Johnson) ...",456,150
9413,204352,Ad Astra,2935510,tt2935510,2019,"Adventure, Drama, Mystery",Astronaut Roy McBride undertakes a mission acr...,53.5,16
9414,204542,It: Chapter Two,7349950,tt7349950,2019,"Drama, Fantasy, Horror",Twenty-seven years after their first encounter...,58,19
9415,204698,Joker,7286456,tt7286456,2019,"Crime, Drama, Thriller","In Gotham City, mentally troubled comedian Art...",221.5,55


# Create the Genres dataset

In [5]:
#list of unique genres
genreset = set()
for i in range(len(movies)):
    genrevec = [genre.strip() for genre in movies.iloc[i,5].split(',')]         #get a list of genre string without whitespace, seperated into individual elements
    genreset.update(genrevec)                                                   #add them to a set to keep unique values
genrelist = list(genreset)                                                      #convert to list



In [7]:
Genres = pd.DataFrame()
for i in range(len(genrelist)):
    Genres.loc[i,0] = genrelist[i]                                              #genre name 
    Genres.loc[i,1] = i                                                         #giving position in the list as genreid
Genres.columns = ['genre','genreID']
# Genres.to_csv('finalDB/final/Genres.csv',index=False)                           #save file



# Create the GenreOfFilm dataset

In [8]:
GenreOfFilm = pd.DataFrame()
for i in range(len(movies)):
    genrevec = [genre.strip() for genre in movies.iloc[i,5].split(',')]          #get a list of genre string without whitespace, seperated into individual elements
    #add each element in the list to the dataframe
    for j in range(len(genrevec)):
        GenreOfFilm.loc[i+j,0] = movies.iloc[i,2]                                #save imdbid 
        genre = genrevec[j]                                                      #retrieve genre name                                    
        GenreOfFilm.loc[i+j,1] = genrelist.index(genre)                          #retrieve genreid which is = position in the list  

GenreOfFilm.columns = ['movieID', 'genreID']        
# GenreOfFilm.to_csv('finalDB/final/GenreOfFilm.csv',index=False)                   #save file    


# Create the Movie dataset

In [17]:
Movie = pd.DataFrame()
for i in range(len(movies)):
    Movie.loc[i,0] = movies.iloc[i,2]                                  #imdbid 
    Movie.loc[i,1] = movies.iloc[i,1]                                  #title 
    year = str(movies.iloc[i,4])
    if len(year) != 4:
        year = ' '
    try:
        int(year)
    except:
        year = ' '
    Movie.loc[i,2] = year                                              #year 
    Movie.loc[i,3] = movies.iloc[i,6]                                  #description 
    Movie.loc[i,4] = movies.iloc[i,7]                                  #total_ratings
    Movie.loc[i,5] = movies.iloc[i,8]                                  #rating_count

Movie.columns = ['movieID', 'title', 'year','description','total_ratings','rating_count']        
# Movie.to_csv('finalDB/final/Movie.csv',index=False)                    #save file    

# Create the Person dataset

In [72]:
cast = pd.read_csv('finalDB/Filmpersonalities_preliminary.csv', header = 0) 
Person = cast.drop(columns=['yob'])
# Person.to_csv('finalDB/final/Person.csv',index=False)                           #save file


# Create the FilmDirector dataset

In [ ]:
cast = pd.read_csv('finalDB/Filmpersonalities_preliminary.csv', header = 0)
title = pd.read_csv('finalDB/title_principals_preliminary.csv', header = 0)   #personID, cast/director, imdbid dataset 
movie_dict = dict.fromkeys(movies.imdb_id,0)                                  #dictionary of movieids 
title_movie_dict = dict.fromkeys(title.imdb_id,0)

FilmCast = pd.DataFrame(columns = ['imdb_id','personID', 'yob'])              #create a new dataframe
FilmDirector = pd.DataFrame(columns = ['imdb_id','personID', 'yob'])          #create a new dataframe

#delete movies that are not in the final movie list 
for movie in title_movie_dict:
    if movie not in movie_dict:
        title = title.drop(title[title.imdb_id == movie].index)
        
#seperating the dataset into cast and director
for i in range(len(title)):                                                   
    #creating FilmCast dataset
    if title.iloc[i,1] == 'cast':           
        FilmCast.loc[i] = title.iloc[i]                                        #add the movieID and personID
    
    #creating FilmDirector dataset
    elif title.iloc[i,1] == 'director':      
        FilmDirector.loc[i] = title.iloc[i]                                    #add the movieID and personID

        
FilmCast.reset_index(inplace=True, drop= True)
FilmCast = FilmCast.drop(columns=['yob'])
FilmCast.columns = ['movieID','personID']

FilmDirector.reset_index(inplace=True, drop= True)
FilmDirector = FilmDirector.drop(columns=['yob'])
FilmDirector.columns = ['movieID','personID']

# FilmCast.to_csv('finalDB/final/FilmCast.csv',index=False)                     #save file
# FilmDirector.to_csv('finalDB/final/FilmDirector.csv',index=False)             #save file